# SW-3-GraphOperations

**Navigation** : [<< 2-RDFBasics](SW-2-RDFBasics.ipynb) | [Index](README.md) | [4-SPARQL >>](SW-4-SPARQL.ipynb)

## Operations sur les graphes RDF

### Duree estimee : 50 minutes

A la fin de ce notebook, vous saurez :
1. **Lire** des fichiers RDF dans differents formats (Turtle, RDF/XML, NTriples)
2. **Ecrire** des graphes RDF vers des fichiers et des chaines
3. **Fusionner** des graphes RDF avec gestion des conflits
4. **Selectionner** des triplets selon differents criteres et avec LINQ
5. **Manipuler** des listes RDF (creer, lire, modifier, supprimer)

### Prerequis
- .NET 9.0 avec .NET Interactive
- Avoir complete [SW-2-RDFBasics](SW-2-RDFBasics.ipynb)

In [ ]:
#r "nuget: dotNetRDF, 3.2.1"

Installing Packages dotNetRDF

In [ ]:
using VDS.RDF;
using VDS.RDF.Parsing;
using VDS.RDF.Writing;
using VDS.RDF.Parsing.Handlers;
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;

Console.WriteLine("dotNetRDF 3.2.1 pret.");

dotNetRDF 3.2.1 pret.


---

## 1. Lecture de fichiers RDF

Les parsers sont dans `VDS.RDF.Parsing`. Chacun implemente `IRdfReader` et supporte `Load(IGraph, ...)` avec un chemin de fichier, un `StreamReader` ou un `TextReader`. Formats supportes : NTriples, Turtle, Notation 3, RDF/XML, JSON-LD, RDF/JSON, RDFa, TriG, NQuads.

In [ ]:
// 1.1 Chargement depuis un fichier Turtle
IGraph g = new Graph();
IGraph h = new Graph();
TurtleParser ttlparser = new TurtleParser();

ttlparser.Load(g, "data/Example.ttl");              // par chemin
ttlparser.Load(h, new StreamReader("data/Example.ttl")); // par stream

Console.WriteLine($"Fichier  : {g.Triples.Count} triplets");
Console.WriteLine($"Stream   : {h.Triples.Count} triplets");
Console.WriteLine($"Identique: {g.Triples.Count == h.Triples.Count}");

Fichier  : 4 triplets


Stream   : 4 triplets


Identique: True


In [ ]:
// 1.2 StringParser (detection auto) vs Parser specifique
Graph g1 = new Graph();
StringParser.Parse(g1, "<http://example.org/a> <http://example.org/b> <http://example.org/c>.");
Console.WriteLine($"StringParser (auto)  : {g1.Triples.Count} triplet");

Graph g2 = new Graph();
new NTriplesParser().Load(g2, new StringReader("<http://example.org/a> <http://example.org/b> <http://example.org/c>."));
Console.WriteLine($"NTriplesParser       : {g2.Triples.Count} triplet");

StringParser (auto)  : 1 triplet


NTriplesParser       : 1 triplet


In [ ]:
// 1.3 Handlers de lecture avancee (pas de chargement memoire)
CountHandler handler = new CountHandler();
new TurtleParser().Load(handler, "data/animals.ttl");
Console.WriteLine($"animals.ttl : {handler.Count} triplets (comptes sans charger en memoire)");

animals.ttl : 51 triplets (comptes sans charger en memoire)


#### Interpretation : Methodes de lecture

| Methode | Avantage | Inconvenient |
|---------|----------|--------------|
| `TurtleParser` + fichier | Format explicite, fiable | Fichier requis |
| `StringParser.Parse()` | Detection auto du format | Peut echouer sur fragments ambigus |
| `NTriplesParser` + `StringReader` | Format explicite, en memoire | Necessite de connaitre le format |
| `CountHandler` | Pas de chargement memoire | Ne stocke pas les triplets |

**Handlers disponibles** : `GraphHandler` (stockage), `CountHandler` (comptage), `WriteHandler` (transformation), `PagingHandler` (par lots), `FilterHandler` (filtrage).

> Les parsers sont **reutilisables** et **thread-safe**.

---

## 2. Ecriture de graphes RDF

Les writers sont dans `VDS.RDF.Writing` et implementent `IRdfWriter`. Formats : NTriples, Turtle, RDF/XML, CompressingTurtle, RDF/JSON, CSV, TSV.

In [ ]:
// 2.1 Comparaison de formats de sortie
IGraph g = new Graph();
new TurtleParser().Load(g, "data/Example.ttl");

Console.WriteLine("=== NTriples ===");
Console.WriteLine(VDS.RDF.Writing.StringWriter.Write(g, new NTriplesWriter()));

Console.WriteLine("=== Turtle Compresse ===");
Console.WriteLine(VDS.RDF.Writing.StringWriter.Write(g, new CompressingTurtleWriter()));

=== NTriples ===


<http://www.w3.org/TR/rdf-syntax-grammar> <http://purl.org/dc/elements/1.1/title> "RDF/XML Syntax Specification (Revised)"^^<http://www.w3.org/2001/XMLSchema#string> .
_:autos3 <http://example.org/stuff/1.0/fullname> "Dave Beckett"^^<http://www.w3.org/2001/XMLSchema#string> .
_:autos3 <http://example.org/stuff/1.0/homePage> <http://purl.org/net/dajobe/> .
<http://www.w3.org/TR/rdf-syntax-grammar> <http://example.org/stuff/1.0/editor> _:autos3 .



=== Turtle Compresse ===


@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix dc: <http://purl.org/dc/elements/1.1/>.
@prefix ex: <http://example.org/stuff/1.0/>.

<http://www.w3.org/TR/rdf-syntax-grammar> ex:editor [ex:fullname "Dave Beckett" ; 
                                                     ex:homePage <http://purl.org/net/dajobe/>];
                                          dc:title "RDF/XML Syntax Specification (Revised)".



In [ ]:
// 2.2 RDF/XML : deux methodes pour obtenir une chaine
var rdfxmlwriter = new RdfXmlWriter();

// Methode 1 : Helper (une ligne)
String data1 = VDS.RDF.Writing.StringWriter.Write(g, rdfxmlwriter);

// Methode 2 : System.IO.StringWriter (controle total)
var sw = new System.IO.StringWriter();
rdfxmlwriter.Save(g, sw);
String data2 = sw.ToString();

Console.WriteLine($"Helper   : {data1.Length} car.");
Console.WriteLine($"StringWriter: {data2.Length} car.");
Console.WriteLine($"Identiques  : {data1 == data2}");
Console.WriteLine($"\n=== RDF/XML (extrait) ===\n{data1.Substring(0, Math.Min(data1.Length, 400))}...");

Helper   : 1017 car.


StringWriter: 1017 car.


Identiques  : True



=== RDF/XML (extrait) ===
<?xml version="1.0" encoding="utf-16"?>
<!DOCTYPE rdf:RDF [
	<!ENTITY rdf 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
	<!ENTITY rdfs 'http://www.w3.org/2000/01/rdf-schema#'>
	<!ENTITY xsd 'http://www.w3.org/2001/XMLSchema#'>
	<!ENTITY dc 'http://purl.org/dc/elements/1.1/'>
	<!ENTITY ex 'http://example.org/stuff/1.0/'>
]>
<rdf:RDF xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:x...


In [ ]:
// 2.3 Configuration avancee des writers
public static string SaveConfigured(IGraph g, IRdfWriter writer)
{
    if (writer is IPrettyPrintingWriter ppw) ppw.PrettyPrintMode = true;
    if (writer is IHighSpeedWriter hsw) hsw.HighSpeedModePermitted = false;
    if (writer is ICompressingWriter cw) cw.CompressionLevel = WriterCompressionLevel.High;
    return VDS.RDF.Writing.StringWriter.Write(g, writer);
}

Console.WriteLine("=== Turtle haute compression ===");
Console.WriteLine(SaveConfigured(g, new CompressingTurtleWriter()));

=== Turtle haute compression ===


@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix dc: <http://purl.org/dc/elements/1.1/>.
@prefix ex: <http://example.org/stuff/1.0/>.

<http://www.w3.org/TR/rdf-syntax-grammar> ex:editor [ex:fullname "Dave Beckett" ; 
                                                     ex:homePage <http://purl.org/net/dajobe/>];
                                          dc:title "RDF/XML Syntax Specification (Revised)".



#### Interpretation : Writers et configuration

| Writer | Format | Lisibilite | Taille |
|--------|--------|------------|--------|
| `NTriplesWriter` | NTriples | Faible | Verbose |
| `CompressingTurtleWriter` | Turtle | Elevee | Compact |
| `RdfXmlWriter` | RDF/XML | Moyenne | Moyenne |

| Interface | Capacite |
|-----------|----------|
| `IPrettyPrintingWriter` | Formatage lisible (indentation) |
| `ICompressingWriter` | Abreviations maximales |
| `IHighSpeedWriter` | Desactive le formatage pour la vitesse |

> Tous les writers sont **reutilisables** et **thread-safe**.

---

## 3. Fusion de graphes

La methode `Merge()` combine deux graphes en respectant la semantique RDF : pas de doublons, renommage des noeuds blancs.

In [ ]:
// 3.1 Fusion de deux graphes
Graph g1 = new Graph();
Graph g2 = new Graph();
var ttlP = new TurtleParser();

ttlP.Load(g1, "data/Example.ttl");
ttlP.Load(g2, "data/animals.ttl");
int somme = g1.Triples.Count + g2.Triples.Count;

Console.WriteLine($"Graphe 1 (Example.ttl) : {g1.Triples.Count} triplets");
Console.WriteLine($"Graphe 2 (animals.ttl) : {g2.Triples.Count} triplets");

g1.Merge(g2);
Console.WriteLine($"Apres fusion           : {g1.Triples.Count} triplets (somme theorique: {somme})");
Console.WriteLine($"Doublons supprimes     : {somme - g1.Triples.Count}");

Graphe 1 (Example.ttl) : 4 triplets


Graphe 2 (animals.ttl) : 51 triplets


Apres fusion           : 55 triplets (somme theorique: 55)


Doublons supprimes     : 0


#### Interpretation

- Les triplets identiques ne sont **pas dupliques** (semantique d'ensemble)
- Les noeuds blancs sont **renommes** pour eviter les collisions
- L'operation est **asymetrique** : `g1.Merge(g2)` modifie `g1`, pas `g2`

---

## 4. Selection de triplets

`IGraph` propose des methodes `GetTriplesWithXxx()` retournant `IEnumerable<Triple>`, composables avec LINQ.

In [ ]:
// 4.1 Selection par predicat et par sujet
Graph g = new Graph();
new TurtleParser().Load(g, "data/animals.ttl");

IUriNode rdfType = g.CreateUriNode("rdf:type");
Console.WriteLine("=== Triplets rdf:type ===");
foreach (Triple t in g.GetTriplesWithPredicate(rdfType))
    Console.WriteLine($"  {t}");

IUriNode rex = g.GetUriNode(new Uri("http://example.org/animals#rex"));
if (rex != null)
{
    Console.WriteLine("\n=== Proprietes de Rex ===");
    foreach (Triple t in g.GetTriplesWithSubject(rex))
        Console.WriteLine($"  {t}");
}

=== Triplets rdf:type ===


  http://example.org/animals#Animal , http://www.w3.org/1999/02/22-rdf-syntax-ns#type , http://www.w3.org/2000/01/rdf-schema#Class


  http://example.org/animals#Mammal , http://www.w3.org/1999/02/22-rdf-syntax-ns#type , http://www.w3.org/2000/01/rdf-schema#Class


  http://example.org/animals#Bird , http://www.w3.org/1999/02/22-rdf-syntax-ns#type , http://www.w3.org/2000/01/rdf-schema#Class


  http://example.org/animals#Dog , http://www.w3.org/1999/02/22-rdf-syntax-ns#type , http://www.w3.org/2000/01/rdf-schema#Class


...


=== Proprietes de Rex ===


  http://example.org/animals#rex , http://www.w3.org/1999/02/22-rdf-syntax-ns#type , http://example.org/animals#Dog


  http://example.org/animals#rex , http://example.org/animals#name , Rex^^http://www.w3.org/2001/XMLSchema#string


  http://example.org/animals#rex , http://example.org/animals#age , 5^^http://www.w3.org/2001/XMLSchema#integer


  http://example.org/animals#rex , http://example.org/animals#sound , Woof^^http://www.w3.org/2001/XMLSchema#string


In [ ]:
// 4.2 Selection combinee et LINQ
IUriNode nameProp = g.CreateUriNode(new Uri("http://example.org/animals#name"));
Console.WriteLine("=== Noms des animaux ===");
foreach (Triple t in g.GetTriplesWithPredicate(nameProp))
    Console.WriteLine($"  {t.Subject} -> {t.Object}");

// LINQ : animaux de plus de 5 ans
IUriNode ageProp = g.CreateUriNode(new Uri("http://example.org/animals#age"));
var older = g.GetTriplesWithPredicate(ageProp)
    .Where(t => t.Object is ILiteralNode lit && int.TryParse(lit.Value, out int age) && age > 5)
    .Select(t => new { Animal = t.Subject, Age = t.Object });

Console.WriteLine("\n=== Animaux > 5 ans (LINQ) ===");
foreach (var a in older)
    Console.WriteLine($"  {a.Animal} : {a.Age}");

=== Noms des animaux ===


  http://example.org/animals#rex -> Rex^^http://www.w3.org/2001/XMLSchema#string


  http://example.org/animals#minou -> Minou^^http://www.w3.org/2001/XMLSchema#string


  http://example.org/animals#coco -> Coco^^http://www.w3.org/2001/XMLSchema#string


  http://example.org/animals#buddy -> Buddy^^http://www.w3.org/2001/XMLSchema#string



=== Animaux > 5 ans (LINQ) ===


  http://example.org/animals#coco : 12^^http://www.w3.org/2001/XMLSchema#integer


  http://example.org/animals#buddy : 7^^http://www.w3.org/2001/XMLSchema#integer


#### Interpretation : Methodes de selection

| Methode | Pattern | Equivalent SPARQL |
|---------|---------|-------------------|
| `GetTriplesWithSubject(s)` | `s ? ?` | `SELECT ?p ?o WHERE { s ?p ?o }` |
| `GetTriplesWithPredicate(p)` | `? p ?` | `SELECT ?s ?o WHERE { ?s p ?o }` |
| `GetTriplesWithObject(o)` | `? ? o` | `SELECT ?s ?p WHERE { ?s ?p o }` |
| `GetTriplesWithSubjectPredicate(s,p)` | `s p ?` | `SELECT ?o WHERE { s p ?o }` |
| `GetTriplesWithPredicateObject(p,o)` | `? p o` | `SELECT ?s WHERE { ?s p o }` |

Les resultats sont des `IEnumerable<Triple>` composables avec LINQ.

---

## 5. Listes RDF

Les listes RDF sont des structures chainees encodees avec `rdf:first` et `rdf:rest`. dotNetRDF fournit des methodes haut niveau dans `VDS.RDF.Extensions`.

In [ ]:
// 5.1 Creer et lire une liste RDF
IGraph g = new Graph();
string rdfData = @"
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ex: <http://example.org/ns#> .
ex:subj ex:pred _:b1 .
_:b1 rdf:first 1 . _:b1 rdf:rest _:b2 .
_:b2 rdf:first 2 . _:b2 rdf:rest _:b3 .
_:b3 rdf:first 3 . _:b3 rdf:rest rdf:nil .";

TurtleParser parser = new TurtleParser();
parser.Load(g, new StringReader(rdfData));

IUriNode predicate = g.CreateUriNode("ex:pred");
INode root = g.GetTriplesWithPredicate(predicate).First().Object;

Console.WriteLine($"GetListItems  : [{string.Join(", ", g.GetListItems(root))}]");
Console.WriteLine($"GetListNodes  : [{string.Join(", ", g.GetListNodes(root))}]");
Console.WriteLine($"GetListAsTriples : {g.GetListAsTriples(root).Count()} triplets");

GetListItems  : [1^^http://www.w3.org/2001/XMLSchema#integer, 2^^http://www.w3.org/2001/XMLSchema#integer, 3^^http://www.w3.org/2001/XMLSchema#integer]


GetListNodes  : [_:b1, _:b2, _:b3]


GetListAsTriples : 6 triplets


#### Interpretation : Structure interne

```
root -> [1 | rest] -> [2 | rest] -> [3 | rest] -> nil
```

| Methode | Retourne | Exemple |
|---------|----------|---------|
| `GetListItems(root)` | Valeurs (`rdf:first`) | `1, 2, 3` |
| `GetListNodes(root)` | Noeuds intermediaires | `_:b1, _:b2, _:b3` |
| `GetListAsTriples(root)` | Tous les triplets | `rdf:first` + `rdf:rest` |

In [ ]:
// 5.2 AssertList et RetractList
INode newRoot = g.AssertList(new List<INode>() { (true).ToLiteral(g), (false).ToLiteral(g) });
Console.WriteLine($"AssertList    : [{string.Join(", ", g.GetListItems(newRoot))}]");

int avant = g.Triples.Count;
g.RetractList(newRoot);
Console.WriteLine($"RetractList   : {avant} -> {g.Triples.Count} triplets");

AssertList    : [true^^http://www.w3.org/2001/XMLSchema#boolean, false^^http://www.w3.org/2001/XMLSchema#boolean]


RetractList   : 11 -> 7 triplets


In [ ]:
// 5.3 AddToList et RemoveFromList
Console.WriteLine($"Avant         : [{string.Join(", ", g.GetListItems(root))}]");

g.AddToList(root, new List<INode>() { (true).ToLiteral(g), (false).ToLiteral(g) });
Console.WriteLine($"AddToList     : [{string.Join(", ", g.GetListItems(root))}]");

g.RemoveFromList(root, new List<INode>() { (true).ToLiteral(g), (false).ToLiteral(g) });
Console.WriteLine($"RemoveFromList: [{string.Join(", ", g.GetListItems(root))}]");

Avant         : [1^^http://www.w3.org/2001/XMLSchema#integer, 2^^http://www.w3.org/2001/XMLSchema#integer, 3^^http://www.w3.org/2001/XMLSchema#integer]


AddToList     : [1^^http://www.w3.org/2001/XMLSchema#integer, 2^^http://www.w3.org/2001/XMLSchema#integer, 3^^http://www.w3.org/2001/XMLSchema#integer, true^^http://www.w3.org/2001/XMLSchema#boolean, false^^http://www.w3.org/2001/XMLSchema#boolean]


RemoveFromList: [1^^http://www.w3.org/2001/XMLSchema#integer, 2^^http://www.w3.org/2001/XMLSchema#integer, 3^^http://www.w3.org/2001/XMLSchema#integer]


#### Interpretation : API des listes

| Operation | Methode | Comportement |
|-----------|---------|--------------|
| **Creer** | `AssertList(elements)` | Genere noeuds blancs + triplets |
| **Supprimer tout** | `RetractList(root)` | Suppression recursive |
| **Ajouter** | `AddToList(root, elements)` | Ajoute en fin de liste |
| **Retirer** | `RemoveFromList(root, elements)` | Supprime toutes les occurrences |

> **Quand utiliser les listes RDF ?** : Collections ordonnees (sequences, etapes). Pour des ensembles non ordonnes, preferez des triplets individuels.

---

## 6. Exercices pratiques

### Exercice 1 : Charger et explorer

Chargez `data/animals.ttl` et affichez : le nombre total de triplets, les triplets `rdf:type`, et les proprietes de `ex:rex`.

In [ ]:
// Exercice 1 : Votre code ici
// 1. Charger animals.ttl avec TurtleParser
// 2. Afficher g.Triples.Count
// 3. GetTriplesWithPredicate pour rdf:type
// 4. GetTriplesWithSubject pour ex:rex

### Exercice 2 : Fusionner et serialiser

Chargez `data/Example.ttl` et `data/animals.ttl`, fusionnez-les, et ecrivez le resultat en Turtle compresse.

In [ ]:
// Exercice 2 : Votre code ici
// 1. Charger deux IGraph
// 2. Merge()
// 3. CompressingTurtleWriter
// 4. Afficher

### Exercice 3 : Liste RDF

Creez une liste `["Alice", "Bob", "Charlie"]` avec `AssertList`, ajoutez `"Diana"`, supprimez `"Bob"`, affichez le resultat.

In [ ]:
// Exercice 3 : Votre code ici
// IGraph g = new Graph();
// var root = g.AssertList(new List<INode>() { "Alice".ToLiteral(g), ... });
// g.AddToList(root, ...) / g.RemoveFromList(root, ...)
// Console.WriteLine(string.Join(", ", g.GetListItems(root)));

---

## Resume

| Section | Concepts cles | APIs principales |
|---------|---------------|------------------|
| **1. Lecture** | Parsers, StringParser, Handlers | `TurtleParser`, `NTriplesParser`, `CountHandler` |
| **2. Ecriture** | Writers, configuration | `NTriplesWriter`, `CompressingTurtleWriter`, `RdfXmlWriter` |
| **3. Fusion** | Merge, deduplication | `IGraph.Merge()` |
| **4. Selection** | GetTriplesWithXxx, LINQ | `GetTriplesWithSubject`, `GetTriplesWithPredicate` |
| **5. Listes** | Collections ordonnees | `AssertList`, `GetListItems`, `AddToList`, `RetractList` |

### Prochaine etape

Dans **SW-4-SPARQL**, nous apprendrons a interroger les graphes RDF avec le langage SPARQL et le QueryBuilder de dotNetRDF.

---

**Navigation** : [<< 2-RDFBasics](SW-2-RDFBasics.ipynb) | [Index](README.md) | [4-SPARQL >>](SW-4-SPARQL.ipynb)